# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [20]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [21]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [22]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [23]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [24]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [25]:
# Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify 
                WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")

#### Set Keyspace

In [26]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create tables to ask the following three questions (queries) on the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [27]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
session.execute("""CREATE TABLE IF NOT EXISTS music_app_history (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession));""")
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
session.execute("""CREATE TABLE IF NOT EXISTS user_app_history (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY (userId, sessionId, itemInSession));""")
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
session.execute("""CREATE TABLE IF NOT EXISTS songplays (song text, firstName text, lastName text, userId int, PRIMARY KEY (song, userId));""")   

In [28]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length) VALUES (%s, %s, %s, %s, %s)"
        query2 = "INSERT INTO user_app_history (userId, sessionId, itemInSession, artist, song, firstName, lastName) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        query3 = "INSERT INTO songplays (song, firstName, lastName, userId) VALUES (%s, %s, %s, %s)"
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query1, (int(sessionId), int(itemInSession), artist, song, float(length)))
        #session.execute(query2, (int(userId), int(sessionId), int(itemInSession), artist, song, firstName, lastName))
        session.execute(query3, (song, firstName, lastName, int(userId)))

InvalidRequest: Error from server: code=2200 [Invalid query] message="Undefined column name userid"

#### Do a SELECT to verify that the data have been inserted into the table

In [14]:
## Perform a SELECT statement to verify the data was entered into the table
rows = session.execute("""SELECT artist, song, length from music_app_history WHERE sessionId = 338 AND itemInSession = 4""")
for (artist, song, length) in rows:
    print(artist, song, length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [15]:
rows = session.execute("""SELECT artist, song, firstName, lastName from user_app_history WHERE userId = 10 AND sessionId = 182""")
for (artist, song, firstname, lastname) in rows:
    print(artist, song, firstname, lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [16]:
rows = session.execute("""SELECT firstName, lastName FROM songplays WHERE song = 'All Hands Against His Own'""")
for row in rows:
    print(row)

InvalidRequest: Error from server: code=2200 [Invalid query] message="PRIMARY KEY column "song" cannot be restricted as preceding column "iteminsession" is not restricted"

### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions

In [ ]:
session.execute("DROP TABLE music_app_history")
session.execute("DROP TABLE user_app_history")
session.execute("DROP TABLE songplays")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()